<a href="https://colab.research.google.com/github/R12942159/DeepLearning/blob/main/DLCV_hw1_p3_deeplabv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch


# Get cuda from GPU device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using: {device}")

Using: cuda


In [3]:
import os


train_paths = '/content/drive/MyDrive/NTU_DLCV/p3_data/train'
img_paths_train = sorted([os.path.join(train_paths, i) for i in os.listdir(train_paths) if i.endswith('.jpg')])
mask_paths_train = sorted([os.path.join(train_paths, i) for i in os.listdir(train_paths) if i.endswith('.png')])

val_paths = '/content/drive/MyDrive/NTU_DLCV/p3_data/validation'
img_paths_val = sorted([os.path.join(val_paths, i) for i in os.listdir(val_paths) if i.endswith('.jpg')])
mask_paths_val = sorted([os.path.join(val_paths, i) for i in os.listdir(val_paths) if i.endswith('.png')])

In [4]:
len(img_paths_train), len(mask_paths_train), len(img_paths_val), len(mask_paths_val)

(2000, 2000, 257, 257)

In [5]:
import torch
import random
import numpy as np
from PIL import Image
from copy import deepcopy
from torchvision.transforms.functional import hflip, vflip, to_tensor, normalize


class LandDataset(torch.utils.data.Dataset):
    def __init__(self, img_paths, mask_paths, transform, mode, augment=False):
        self.img_paths = img_paths
        self.mask_paths = mask_paths
        self.transform = transform
        self.mode = mode

        # Data Augmentation
        def original(x): return x
        if augment:
            self.augment = [original, hflip, vflip]
        else:
            self.augment = [original]

        # double check the len of img and mask.
        assert len(self.img_paths) == len(self.mask_paths)

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        # get img path
        img_path = self.img_paths[idx]
        # Read img
        img = Image.open(img_path).convert('RGB')
        # Normalize Image with VGG16's mean and std
        # (H, W, C) -> (C, H, W)
        img = self.transform(img)

        if self.mode != 'test':
            # get mask path
            mask_path = self.mask_paths[idx]
            mask = Image.open(mask_path).convert('RGB')
            mask = np.array(mask)
            # Binarize mask from [0~255] to (0 or 1)
            mask = (mask >= 128).astype(int)

            # squeeze [a,b,c] into [x,y] with 7 category(6 classes + 1 background)
            mask = 100 * mask[:, :, 0] + 10 * mask[:, :, 1] + 1 * mask[:, :, 2]
            raw_mask = deepcopy(mask) # Only perform numerical conversion on the most original data
            mask[raw_mask == 11] = 0  # (Cyan: 011) Urban land
            mask[raw_mask == 110] = 1  # (Yellow: 110) Agriculture land
            mask[raw_mask == 101] = 2  # (Purple: 101) Rangeland
            mask[raw_mask == 10] = 3  # (Green: 010) Forest land
            mask[raw_mask == 1] = 4  # (Blue: 001) Water
            mask[raw_mask == 111] = 5  # (White: 111) Barren land
            mask[raw_mask == 0] = 6  # (Black: 000) Unknown
            mask = torch.tensor(mask)

            # random Data Augmentation
            augmentor = random.choice(self.augment)
            img = augmentor(img)
            mask = augmentor(mask)

            # mask = mask.to(torch.float)  # 将目标标签转换为浮点数类型
            return img, mask
        else:
            return img

In [6]:
import torchvision.transforms as tr


# IMG_SIZE : 512*512
BATCH_SIZE = 4

# https://pytorch.org/vision/main/models/generated/torchvision.models.segmentation.deeplabv3_resnet50.html#torchvision.models.segmentation.deeplabv3_resnet50
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

train_ds = LandDataset(img_paths_train,
                       mask_paths_train,
                       transform = tr.Compose([
                          tr.ToTensor(),
                          tr.Normalize(mean=mean, std=std),
                          ]),
                       mode = 'train',
                       augment = True,)
val_ds = LandDataset(img_paths_val,
                     mask_paths_val,
                     transform = tr.Compose([
                          tr.ToTensor(),
                          tr.Normalize(mean=mean, std=std),
                          ]),
                     mode = 'val',
                     augment = False,)

# num_workers > 0: accelerate loading data by muli-process
train_loader = torch.utils.data.DataLoader(train_ds, BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_ds, BATCH_SIZE*2, shuffle=False, num_workers=4)

In [7]:
from torchvision.models.segmentation.deeplabv3 import deeplabv3_resnet50, deeplabv3_resnet101, DeepLabHead, FCNHead


# https://pytorch.org/hub/pytorch_vision_deeplabv3_resnet101/
# output['out'] contains the semantic masks, and output['aux'] contains the auxiliary loss values per-pixel
# https://pytorch.org/vision/main/_modules/torchvision/models/segmentation/deeplabv3.html#deeplabv3_resnet50
def deeplabv3(mode, num_classes=7):
    if mode == 'resnet101':
        model =  deeplabv3_resnet101(pretrained=True)
        model.aux_classifier = FCNHead(1024, num_classes)
        # DeepLabHead在处理复杂场景、小物体和物体边缘时通常表现更好,
        # ASPP-Conv2d(256, 256, 3, padding=1, bias=False)-nn.BatchNorm2d(256)-nn.ReLU()-nn.Conv2d(256, num_classes, 1)
        model.classifier = DeepLabHead(2048, num_classes)
    elif mode == 'resnet50':
        model = deeplabv3_resnet50(pretrained=True)
        model.aux_classifier = FCNHead(1024, num_classes)
        model.classifier = DeepLabHead(2048, num_classes)

    return model

In [8]:
def mean_iou_score(pred, labels, num_classes=6):
    '''
    Compute mean IoU score over 6 classes
    '''
    mean_iou = []
    for i in range(num_classes):
        tp_fp = torch.sum(pred == i)
        tp_fn = torch.sum(labels == i)
        tp = torch.sum((pred == i) * (labels == i))
        # avoid nan
        if (tp_fp + tp_fn - tp) == 0:
            iou = 0.
        else:
            iou = tp / (tp_fp + tp_fn - tp)
        mean_iou.append(iou)

    return sum(mean_iou) / len(mean_iou)

In [9]:
from tqdm import tqdm


def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) # number of samples
    num_batches = len(dataloader) # batches per epoch
    model.train() # to training mode.
    epoch_loss, epoch_iou = 0, 0
    for batch_i, (x, y) in enumerate(tqdm(dataloader, leave=False)):
        x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True) # move data to GPU

        # Compute prediction loss
        pred = model(x)
        pred_out, pred_aux = pred['out'], pred['aux']
        loss = loss_fn(pred_out, y) + loss_fn(pred_aux, y)
        # Optimization by gradients
        optimizer.zero_grad() # set prevision gradient to 0
        loss.backward() # backpropagation to compute gradients
        optimizer.step() # update model params
        # scheduler.step()

        pred_out = torch.argmax(pred_out, dim=1) # 沿着通道维度选择具有最高分数的通道
        pred_out = pred_out.to(torch.float)  # 将目标标签转换为浮点数类型
        # log
        epoch_loss += loss.item() # tensor -> python value
        epoch_iou += mean_iou_score(pred_out, y).item()

    # return avg loss of epoch, iou of epoch
    return epoch_loss/num_batches, epoch_iou/num_batches


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset) # number of samples
    num_batches = len(dataloader) # batches per epoch

    model.eval() # model to test mode.
    epoch_loss = 0
    epoch_iou = 0
    # No gradient for test data
    with torch.no_grad():
        for batch_i, (x, y) in enumerate(dataloader):
            x, y = x.to(device), y.to(device)

            # Compute prediction loss
            pred = model(x)
            pred = pred['out']
            loss = loss_fn(pred, y)

            pred = torch.argmax(pred, dim=1) # 沿着通道维度选择具有最高分数的通道
            pred = pred.to(torch.float)  # 将目标标签转换为浮点数类型

            # write to logs
            epoch_loss += loss.item()
            epoch_iou += mean_iou_score(pred, y).item()

    return epoch_loss/num_batches, epoch_iou/num_batches

#### Cross Entropy Loss

\begin{align}
L_{ce,i} \quad = \quad
\begin{cases}
  & -log(p_i)   \qquad, \text{ if } y_i= 1\\
  & -log(1-p_i) \; \;    ,\text{ if } y_i=0
\end{cases}
    \end{align}
#### Focal Loss

\begin{align}
L_{focal,i} \quad = \quad
\begin{cases}
  & -\alpha (1-p_i)^rlog(p_i)   \qquad     ,\text{ if } y_i= 1\\
  & -(1-\alpha)p_i^rlog(1-p_i)  \ \ \ \,   ,\text{ if } y_i=0
\end{cases}
    \end{align}

In [10]:
# ref. https://blog.csdn.net/qq_39732684/article/details/105794637
from torch import nn


class focal_loss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2): # Assume that the positive and negative sample ratio is 3:1
        super(focal_loss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.CrossEntropyLoss = nn.CrossEntropyLoss(ignore_index=6) # 用于指定在计算损失时应忽略的类别索引,参数被设置为背景类别的索引，以确保这些像素不会影响模型的训练

    def forward(self, preds, labels):
        Lce = -self.CrossEntropyLoss(preds, labels) # log(pi) or log(1-pi)
        Lfocal = -1 * (self.alpha) * ((1-torch.exp(Lce)) ** self.gamma) * Lce

        return Lfocal

In [11]:
from torch.optim.lr_scheduler import StepLR


EPOCHS = 100
logs = {
    'train_loss': [], 'val_loss': [],
    'train_mean_iou': [], 'val_mean_iou': [],
}

model = deeplabv3(mode='resnet101')
model = model.to(device)
loss_fn = focal_loss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
# citaion: Encoder-Decoder with Atrous Separable Convolution for Semantic Image Segmentation(Liang-Chieh Chen, Yukun Zhu, George Papandreou, Florian Schroff, and Hartwig Adam)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.05, momentum=0.9, weight_decay=4e-5)
# scheduler = StepLR(optimizer, step_size=2, gamma=0.94) # 每2个周期衰减一次, 衰减率为0.94


# Earlystopping
patience = 5
counter = 0
best_IOU = 0

for epoch in tqdm(range(EPOCHS)):
    train_loss, train_mean_iou = train(train_loader, model, loss_fn, optimizer)
    val_loss, val_mean_iou = test(val_loader, model, loss_fn)
    logs['train_loss'].append(train_loss)
    logs['val_loss'].append(val_loss)
    logs['train_mean_iou'].append(train_mean_iou)
    logs['val_mean_iou'].append(val_mean_iou)

    print(f'EPOCH: {(epoch+1):04d} train_loss: {train_loss:.4f} val_loss: {val_loss:.4f}, train_mean_iou: {train_mean_iou:.3f}, val_mean_iou: {val_mean_iou:.3f}')

    # On epoch end
    torch.save(model.state_dict(), "deeplabv3_restnet101_lastmodel.pth")
    # check improvement
    if best_IOU < val_mean_iou:
        counter = 0
        best_IOU = val_mean_iou
        torch.save(model.state_dict(), "deeplabv3_restnet101_BestIOU.pth")
        print('Best_IOU saved!')
    else:
        counter += 1
    if counter >= patience:
        print("Earlystop!")
        break

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/deeplabv3_resnet101_coco-586e9e4e.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet101_coco-586e9e4e.pth
100%|██████████| 233M/233M [00:00<00:00, 374MB/s]
100%|██████████| 500/500 [05:46<00:00,  1.62it/s]
                                                 

EPOCH: 0001 train_loss: 0.5359 val_loss: 0.0633, train_mean_iou: 0.230, val_mean_iou: 0.380


  1%|          | 1/100 [06:48<11:14:31, 408.80s/it]

Best_IOU saved!



100%|██████████| 500/500 [05:08<00:00,  1.62it/s]
                                                 

...

EPOCH: 0039 train_loss: 0.0204 val_loss: 0.0182, train_mean_iou: 0.478, val_mean_iou: 0.507


100%|██████████| 500/500 [05:08<00:00,  1.62it/s]
                                                 

EPOCH: 0040 train_loss: 0.0189 val_loss: 0.0065, train_mean_iou: 0.488, val_mean_iou: 0.568


 39%|███▉      | 39/100 [3:35:43<5:37:24, 331.88s/it]

Earlystop!
